# Preparação de dados 

Esses dados foram utilizados para uma breve previsão com o mask-rcnn

obs: código feito para auxiliar na compreensão do dataset COCO e mask rcnn (básica previsão dos dados, com apenas uma imagem de exemplo)

In [ ]:
import json
import matplotlib.pyplot as plt 
import matplotlib.patches as patches
import PIL 
from PIL import Image
import os
import cv2
import urllib.request
import glob
import numpy as np
import matplotlib.pyplot as plt




Lendo o json

In [ ]:
path = open("./annotations/instances_train2017.json", "r")
anns = json.load(path)

In [ ]:
print(anns['categories']['name'])

## Bicycle_train

In [ ]:
files = glob.glob("./images/train/bicycle_train/*.jpg")

Criando as variáveis:
  - train_image_bicycle:   contém as imagens que vamos utilizar 
  - file_name_bicycle :    contém os nomes dos arquivos 

In [ ]:
train_image_bicycle = []
filename_bicycle=[]

#utilizando o for para percorrer a pasta bicycle_train e pegar o nome de cada imagem
for myFile in files:
    image = cv2.imread(myFile)
    #convertendo as imagens para RGB
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    #adicionando a imagem na variável train_image_bicycle
    train_image_bicycle.append(image)
    #adicionando o nome da imagem no filename_bicycle
    filename_bicycle.append(os.path.basename(myFile))

train_image_bicycle = np.array(train_image_bicycle, dtype=object)
train_image_bicycle.shape
#plt.imshow(train_image[7])
print(len(train_image_bicycle)) 

Criando a variável:
 - id_image_bicycle = para obter o id da imagem contido em filename_bicycle

In [ ]:
path = open("./annotations/instances_train2017.json", "r")
anns = json.load(path)
print(anns)
id_image_bicycle=[]

#fazendo um loop que contem todas as anotações e adicionando em id_image apenas o id da imagem(image_id na anotação json)
for i in range(len(anns['images'])):
  #vendo todos os file_names de image
  filename_bicycle = (anns['images'][i]['file_name'])

  #pegando apenas o id da imagem
  for j in filename_bicycle:
    if filename_bicycle == j:
      id_image_bicycle.append(anns['images'][i]['id'])

print(id_image_bicycle)  

Criando a variável:
 - train_label_bicycle: que contém todas as anotações das imagens
  
      -> Obs: train_label_bicycle é um vetor que abriga um dicionário representando cada anotação

In [ ]:
train_label_bicycle= []
anotacoes = {}

#Este for está percorrendo todas as anotações de treino
for i in range(len(anns['annotations'])):
    #Este percorre os ids das imagens
    for j in id_image_bicycle:
        #loop para ver se o image_id da anotação corresponde ao id da imagem
        if (anns['annotations'][i]['image_id']) == j:
            anotacoes = dict(anns['annotations'][i])
            train_label_bicycle.append(anotacoes)
      
#print(train_label_bicycle)

Criando as variáveis:
 - bbox_bicycle: é um dicionário que contém os bbox presentes em cada imagem
 - id_cat_bicycle : é um dicionário que contém os ids das categorias de cada imagem
 - nome_cat_bicycle: é um dicionário que contém os nomes das categorias de cada imagem

In [ ]:
bbox_bicycle       = {}
id_cat_bicycle     = {}
nome_cat_bicycle   = {}

'''
  uma imagem possui várias anotações, e elas estão em diferentes lugares do json,
  e é possivel identificar qual anotação representa determinada img com o id
  por isso será necessário dois "for":
   - no 1° ele passa pelos id das imagens
   - no 2° ele passa pelas anotações 
  dessa forma é possível deixar todas as informações das imagens juntas
'''
for id in id_image_bicycle:
  #criando um vetor à cada inicio de um loop
  bbox_vetor      = []
  id_cat_vetor    = []
  nome_cat_vetor  = []

  for anot in train_label_bicycle:
    if id == anot['image_id']:
      #fazendo um vetor com todos os BBOX da img x
      bbox_vetor.append(anot['bbox'])

      #fazendo um vetor com todos os IDS DAS CATEGORIAS da img x
      id_cat_vetor.append(int(anot['category_id']))
      #print(idCategoria)

      #fazendo um vetor com todos os NOMES DAS CATEGORIAS da img x
    for x in range(len(id_cat_vetor)):
      nome_cat_vetor.append(anns['categories'][x-1]['name'])
 

  bbox_bicycle[id]     = tuple(bbox_vetor)
  id_cat_bicycle[id]   = tuple(id_cat_vetor)
  nome_cat_bicycle[id] = tuple(nome_cat_vetor)


print((bbox_bicycle[74]))
print((id_cat_bicycle[74]))
print((nome_cat_bicycle[74]))

Aqui é apenas um teste para entender melhor como plotar a imagem com os bbox

In [ ]:
images_path = './BD/images/train/bicycle_train/'
#Colocando o id da imagem que quero que apareça e completando com zeros, até obter 12 digitos
image_name = str(1722).zfill(12)+".jpg" 
image = Image.open(images_path+image_name)

ax = plt.subplots(figsize=(10, 10))


for coord in bbox_bicycle[1722]:
  #fazendo a caixa do bbox
  bb = patches.Rectangle((coord[0], coord[1]), coord[2], coord[3], linewidth=3, edgecolor="blue", facecolor="None")
  ax.add_patch(bb)
  print(bb)

ax.imshow(image)
plt.show()

treinando com mask-rcnn

utilizando apenas uma imagem para melhor compreensão de como funciona este modelo

In [ ]:
import mrcnn
import mrcnn.config
import mrcnn.model
import mrcnn.visualize
import cv2
import os

CLASS_NAMES = ['BG', 'person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush']

class SimpleConfig(mrcnn.config.Config):
    NAME = "coco_inference"
    
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

    NUM_CLASSES = len(CLASS_NAMES)

model = mrcnn.model.MaskRCNN(mode="inference", 
                             config=SimpleConfig(),
                             model_dir=os.getcwd())

model.load_weights(filepath="mask_rcnn_coco.h5", 
                   by_name=True)

image = cv2.imread("000000000074.jpg")
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

r = model.detect([image], verbose=0)

r = r[0]

'''
rois: As caixas ao redor de cada objeto detectado.
class_ids: os IDs de classe para os objetos.
scores: As pontuações de classe para cada objeto.
masks: As máscaras.

'''

#chamando a imagem para ser plotada
mrcnn.visualize.display_instances(image=image, 
                                  boxes=r['rois'], 
                                  masks=r['masks'], 
                                  class_ids=r['class_ids'], 
                                  class_names=CLASS_NAMES, 
                                  scores=r['scores'])

# Bus_train

In [ ]:
files = glob.glob("./BD/images/train/bus_train/*.jpg")

In [ ]:
train_image_bus = []
filename_bus=[]

#utilizando o for para percorrer a pasta bus_train e pegar o nome de cada imagem
for myFile in files:
    image = cv2.imread(myFile)
    #convertendo as imagens para RGB
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    #adicionando a imagem na variável train_image_bus
    train_image_bus.append(image)
    #adicionando o nome da imagem no filename_bus
    filename_bus.append(os.path.basename(myFile))

train_image_bus = np.array(train_image_bus, dtype=object)
train_image_bus.shape
#plt.imshow(train_image_bus[7])
print(len(train_image_bus)) 

In [ ]:
id_image_bus=[]

#fazendo um loop que contem todas as anotações e adicionando em id_image apenas o id da imagem(image_id na anotação json)
for i in range(len(anns['images'])):
  #vendo todos os file_names de image
    filename_bus = (anns['images'][i]['file_name'])

    #pegando apenas o id da imagem
    for j in filename_bus:
      if filename_bus == j:
          id_image_bus.append(anns['images'][i]['id'])

#print(id_image_bus)  

In [ ]:
train_label_bus= []
anotacoes = {}

#Este for está percorrendo todas as anotações de treino
for i in range(len(anns['annotations'])):
    #Este percorre os ids das imagens
    for j in id_image_bus:
        #loop para ver se o image_id da anotação corresponde ao id da imagem
        if (anns['annotations'][i]['image_id']) == j:
            anotacoes = dict(anns['annotations'][i])
            train_label_bus.append(anotacoes)
      
#print(train_label_bus)

In [ ]:
bbox_bus       = {}
id_cat_bus     = {}
nome_cat_bus   = {}

'''
  uma imagem possui várias anotações, e elas estão em diferentes lugares do json,
  e é possivel identificar qual anotação representa determinada img com o id
  por isso será necessário dois "for":
   - no 1° ele passa pelos id das imagens
   - no 2° ele passa pelas anotações 
  dessa forma é possível deixar todas as informações das imagens juntas
'''
for id in id_image_bus:
  #criando um vetor à cada inicio de um loop
  bbox_vetor      = []
  id_cat_vetor    = []
  nome_cat_vetor  = []

  for anot in train_label_bus:
    if id == anot['image_id']:
      #fazendo um vetor com todos os BBOX da img x
      bbox_vetor.append(anot['bbox'])

      #fazendo um vetor com todos os IDS DAS CATEGORIAS da img x
      id_cat_vetor.append(int(anot['category_id']))
      #print(idCategoria)

      #fazendo um vetor com todos os NOMES DAS CATEGORIAS da img x
    for x in range(len(id_cat_vetor)):
      nome_cat_vetor.append(anns['categories'][x-1]['name'])
 

  bbox_bus[id]     = tuple(bbox_vetor)
  id_cat_bus[id]   = tuple(id_cat_vetor)
  nome_cat_bus[id] = tuple(nome_cat_vetor)


print((bbox_bus[74]))
print((id_cat_bus[74]))
print((nome_cat_bus[74]))

# Car_train

In [ ]:
files = glob.glob("./BD/images/train/car_train/*.jpg")

In [ ]:
train_image_car = []
filename_car=[]

#utilizando o for para percorrer a pasta car_train e pegar o nome de cada imagem
for myFile in files:
    image = cv2.imread(myFile)
    #convertendo as imagens para RGB
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    #adicionando a imagem na variável train_image_car
    train_image_car.append(image)
    #adicionando o nome da imagem no filename_car
    filename_car.append(os.path.basename(myFile))

train_image_car = np.array(train_image_car, dtype=object)
train_image_car.shape
#plt.imshow(train_image_car[7])
print(len(train_image_car)) 

In [ ]:
id_image_car=[]

#fazendo um loop que contem todas as anotações e adicionando em id_image apenas o id da imagem(image_id na anotação json)
for i in range(len(anns['images'])):
  #vendo todos os file_names de image
    filename_car = (anns['images'][i]['file_name'])

    #pegando apenas o id da imagem
    for j in filename_car:
      if filename_car == j:
          id_image_car.append(anns['images'][i]['id'])

#print(id_image_car)  

In [ ]:
train_label_car= []
anotacoes = {}

#Este for está percorrendo todas as anotações de treino
for i in range(len(anns['annotations'])):
    #Este percorre os ids das imagens
    for j in id_image_car:
        #loop para ver se o image_id da anotação corresponde ao id da imagem
        if (anns['annotations'][i]['image_id']) == j:
            anotacoes = dict(anns['annotations'][i])
            train_label_car.append(anotacoes)
      
#print(train_label_car)

In [ ]:
bbox_car       = {}
id_cat_car     = {}
nome_cat_car   = {}

'''
  uma imagem possui várias anotações, e elas estão em diferentes lugares do json,
  e é possivel identificar qual anotação representa determinada img com o id
  por isso será necessário dois "for":
   - no 1° ele passa pelos id das imagens
   - no 2° ele passa pelas anotações 
  dessa forma é possível deixar todas as informações das imagens juntas
'''
for id in id_image_car:
  #criando um vetor à cada inicio de um loop
  bbox_vetor      = []
  id_cat_vetor    = []
  nome_cat_vetor  = []

  for anot in train_label_car:
    if id == anot['image_id']:
      #fazendo um vetor com todos os BBOX da img x
      bbox_vetor.append(anot['bbox'])

      #fazendo um vetor com todos os IDS DAS CATEGORIAS da img x
      id_cat_vetor.append(int(anot['category_id']))
      #print(idCategoria)

      #fazendo um vetor com todos os NOMES DAS CATEGORIAS da img x
    for x in range(len(id_cat_vetor)):
      nome_cat_vetor.append(anns['categories'][x-1]['name'])
 

  bbox_car[id]     = tuple(bbox_vetor)
  id_cat_car[id]   = tuple(id_cat_vetor)
  nome_cat_car[id] = tuple(nome_cat_vetor)


print((bbox_car[74]))
print((id_cat_car[74]))
print((nome_cat_car[74]))

# Motorcycle_train

In [ ]:
files = glob.glob("./BD/images/train/motorcycle_train/*.jpg")

In [ ]:
train_image_motorcycle = []
filename_motorcycle=[]

#utilizando o for para percorrer a pasta motorcycle_train e pegar o nome de cada imagem
for myFile in files:
    image = cv2.imread(myFile)
    #convertendo as imagens para RGB
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    #adicionando a imagem na variável train_image_motorcycle
    train_image_motorcycle.append(image)
    #adicionando o nome da imagem no filename_motorcycle
    filename_motorcycle.append(os.path.basename(myFile))

train_image_motorcycle = np.array(train_image_motorcycle, dtype=object)
train_image_motorcycle.shape
#plt.imshow(train_image_motorcycle[7])
print(len(train_image_motorcycle)) 

In [ ]:
id_image_motorcycle=[]

#fazendo um loop que contem todas as anotações e adicionando em id_image apenas o id da imagem(image_id na anotação json)
for i in range(len(anns['images'])):
  #vendo todos os file_names de image
    filename_motorcycle = (anns['images'][i]['file_name'])

    #pegando apenas o id da imagem
    for j in filename_motorcycle:
      if filename_motorcycle == j:
          id_image_motorcycle.append(anns['images'][i]['id'])

#print(id_image_motorcycle)  

In [ ]:
train_label_motorcycle= []
anotacoes = {}

#Este for está percorrendo todas as anotações de treino
for i in range(len(anns['annotations'])):
    #Este percorre os ids das imagens
    for j in id_image_motorcycle:
        #loop para ver se o image_id da anotação corresponde ao id da imagem
        if (anns['annotations'][i]['image_id']) == j:
            anotacoes = dict(anns['annotations'][i])
            train_label_motorcycle.append(anotacoes)
      
#print(train_label_motorcycle)

In [ ]:
bbox_motorcycle      = {}
id_cat_motorcycle    = {}
nome_cat_motorcycle  = {}

'''
  uma imagem possui várias anotações, e elas estão em diferentes lugares do json,
  e é possivel identificar qual anotação representa determinada img com o id
  por isso será necessário dois "for":
   - no 1° ele passa pelos id das imagens
   - no 2° ele passa pelas anotações 
  dessa forma é possível deixar todas as informações das imagens juntas
'''
for id in id_image_motorcycle:
  #criando um vetor à cada inicio de um loop
  bbox_vetor      = []
  id_cat_vetor    = []
  nome_cat_vetor  = []

  for anot in train_label_motorcycle:
    if id == anot['image_id']:
      #fazendo um vetor com todos os BBOX da img x
      bbox_vetor.append(anot['bbox'])

      #fazendo um vetor com todos os IDS DAS CATEGORIAS da img x
      id_cat_vetor.append(int(anot['category_id']))
      #print(idCategoria)

      #fazendo um vetor com todos os NOMES DAS CATEGORIAS da img x
    for x in range(len(id_cat_vetor)):
      nome_cat_vetor.append(anns['categories'][x-1]['name'])
 

  bbox_motorcycle[id]     = tuple(bbox_vetor)
  id_cat_motorcycle[id]   = tuple(id_cat_vetor)
  nome_cat_motorcycle[id] = tuple(nome_cat_vetor)


print((bbox_motorcycle[74]))
print((id_cat_motorcycle[74]))
print((nome_cat_motorcycle[74]))

# Truck_train

In [ ]:
files = glob.glob("./BD/images/train/truck_train/*.jpg")

In [ ]:
train_image_truck = []
filename_truck=[]

#utilizando o for para percorrer a pasta truck_train e pegar o nome de cada imagem
for myFile in files:
    image = cv2.imread(myFile)
    #convertendo as imagens para RGB
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    #adicionando a imagem na variável train_image_truck
    train_image_truck.append(image)
    #adicionando o nome da imagem no filename_truck
    filename_truck.append(os.path.basename(myFile))

train_image_truck = np.array(train_image_truck, dtype=object)
train_image_truck.shape
#plt.imshow(train_image_truck[7])
print(len(train_image_truck)) 

In [ ]:
id_image_truck=[]

#fazendo um loop que contem todas as anotações e adicionando em id_image apenas o id da imagem(image_id na anotação json)
for i in range(len(anns['images'])):
  #vendo todos os file_names de image
    filename_truck = (anns['images'][i]['file_name'])

    #pegando apenas o id da imagem
    for j in filename_truck:
      if filename_truck == j:
          id_image_truck.append(anns['images'][i]['id'])

#print(id_image_truck)  

In [ ]:
train_label_truck= []
anotacoes = {}

#Este for está percorrendo todas as anotações de treino
for i in range(len(anns['annotations'])):
    #Este percorre os ids das imagens
    for j in id_image_truck:
        #loop para ver se o image_id da anotação corresponde ao id da imagem
        if (anns['annotations'][i]['image_id']) == j:
            anotacoes = dict(anns['annotations'][i])
            train_label_truck.append(anotacoes)
      
#print(train_label_truck)

In [ ]:
bbox_truck       = {}
id_cat_truck     = {}
nome_cat_truck   = {}

'''
  uma imagem possui várias anotações, e elas estão em diferentes lugares do json,
  e é possivel identificar qual anotação representa determinada img com o id
  por isso será necessário dois "for":
   - no 1° ele passa pelos id das imagens
   - no 2° ele passa pelas anotações 
  dessa forma é possível deixar todas as informações das imagens juntas
'''
for id in id_image_truck:
  #criando um vetor à cada inicio de um loop
  bbox_vetor      = []
  id_cat_vetor    = []
  nome_cat_vetor  = []

  for anot in train_label_truck:
    if id == anot['image_id']:
      #fazendo um vetor com todos os BBOX da img x
      bbox_vetor.append(anot['bbox'])

      #fazendo um vetor com todos os IDS DAS CATEGORIAS da img x
      id_cat_vetor.append(int(anot['category_id']))
      #print(idCategoria)

      #fazendo um vetor com todos os NOMES DAS CATEGORIAS da img x
    for x in range(len(id_cat_vetor)):
      nome_cat_vetor.append(anns['categories'][x-1]['name'])
 

  bbox_truck[id]     = tuple(bbox_vetor)
  id_cat_truck[id]   = tuple(id_cat_vetor)
  nome_cat_truck[id] = tuple(nome_cat_vetor)


print((bbox_truck[74]))
print((id_cat_truck[74]))
print((nome_cat_truck[74]))

# Person_train

In [ ]:
files = glob.glob("./BD/images/train/person_train/*.jpg")

In [ ]:
train_image_person = []
filename_person=[]

#utilizando o for para percorrer a pasta person_train e pegar o nome de cada imagem
for myFile in files:
    image = cv2.imread(myFile)
    #convertendo as imagens para RGB
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    #adicionando a imagem na variável train_image_person
    train_image_person.append(image)
    #adicionando o nome da imagem no filename_person
    filename_person.append(os.path.basename(myFile))

train_image_person = np.array(train_image_person, dtype=object)
train_image_person.shape
#plt.imshow(train_image_person[7])
print(len(train_image_person)) 

In [ ]:
id_image_person=[]

#fazendo um loop que contem todas as anotações e adicionando em id_image apenas o id da imagem(image_id na anotação json)
for i in range(len(anns['images'])):
  #vendo todos os file_names de image
    filename_person = (anns['images'][i]['file_name'])

    #pegando apenas o id da imagem
    for j in filename_person:
      if filename_person == j:
          id_image_person.append(anns['images'][i]['id'])

#print(id_image_person)  

In [ ]:
train_label_person= []
anotacoes = {}

#Este for está percorrendo todas as anotações de treino
for i in range(len(anns['annotations'])):
    #Este percorre os ids das imagens
    for j in id_image_person:
        #loop para ver se o image_id da anotação corresponde ao id da imagem
        if (anns['annotations'][i]['image_id']) == j:
            anotacoes = dict(anns['annotations'][i])
            train_label_person.append(anotacoes)
      
#print(train_label_person)

In [ ]:
bbox_person       = {}
id_cat_person     = {}
nome_cat_person   = {}

'''
  uma imagem possui várias anotações, e elas estão em diferentes lugares do json,
  e é possivel identificar qual anotação representa determinada img com o id
  por isso será necessário dois "for":
   - no 1° ele passa pelos id das imagens
   - no 2° ele passa pelas anotações 
  dessa forma é possível deixar todas as informações das imagens juntas
'''
for id in id_image_person:
  #criando um vetor à cada inicio de um loop
  bbox_vetor      = []
  id_cat_vetor    = []
  nome_cat_vetor  = []

  for anot in train_label_person:
    if id == anot['image_id']:
      #fazendo um vetor com todos os BBOX da img x
      bbox_vetor.append(anot['bbox'])

      #fazendo um vetor com todos os IDS DAS CATEGORIAS da img x
      id_cat_vetor.append(int(anot['category_id']))
      #print(idCategoria)

      #fazendo um vetor com todos os NOMES DAS CATEGORIAS da img x
    for x in range(len(id_cat_vetor)):
      nome_cat_vetor.append(anns['categories'][x-1]['name'])
 

  bbox_person[id]     = tuple(bbox_vetor)
  id_cat_person[id]   = tuple(id_cat_vetor)
  nome_cat_person[id] = tuple(nome_cat_vetor)


print((bbox_person[74]))
print((id_cat_person[74]))
print((nome_cat_person[74]))